In [1]:
!pip install openai-agents

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 237.6/237.6 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.7/150.7 kB 3.7 MB/s eta 0:00:00


In [2]:
import os
from google.colab import userdata
from google.colab import drive

drive_mount_path = userdata.get('drive_mount_path')
drive.mount(drive_mount_path)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

In [4]:
from agents import Agent, Runner, FileSearchTool, WebSearchTool,ModelSettings
import asyncio
import pandas as pd
import numpy as np
from openai import OpenAI
import json
from pydantic import BaseModel

In [5]:
## EDA Agent
## --- Conduct exploratory data analysis based on available data.
## Decide what variable to use and feature engineering.
## --- Available data are in csv format


client = OpenAI()

# class feature_analysis_result(BaseModel):
#   likely_targets:list[str]
#   selected_features:list[str]
#   feature_engineering:list[str]
#   drop_columns:list[str]
#   missing_value_handling:AgentOutputSchema(dict, strict_json_schema=False)

class feature_analysis_Agent:

  '''
  ## Feature Analysis Agent
  ## --- Conduct exploratory data analysis based on available data.
  ## Decide what variable to use and feature engineering.
  ## --- Available data are in csv format

  '''

  def __init__(self, model:str = 'gpt-4o', user_instructions = '', user_defined_target = ''):
    self.model = model
    self.name = 'Feature Analysis Agent'
    self.user_defined_target = user_defined_target
    self.user_instructions = 'You are a data scientist specialized in feature analysis.\
    You analyze data, gives helpful insights on what variable to use and feature engineering.\
    Then you write code to apply your feature engineering suggesions and transform the dataset. '+user_instructions
    if self.user_defined_target:
      self.user_instructions += f'The target variable is {self.user_defined_target}'
    self.agent = Agent(
        name = self.name,
        model = self.model,
        instructions = self.user_instructions,
        model_settings = ModelSettings(temperature = 0),
        # output_type = feature_analysis_result
        )

  async def run(self, csv_path: str, varb_info_path:str=False):
    df = self._load_csv(csv_path)
    profile = self._profile_data(df)
    suggestions = await self._llm_interpretation(profile, varb_info_path)
    return {
        'raw_profile':profile,
        'Feature_analysis_suggestions':suggestions}

  def _load_csv(self, csv_path:str):
    '''load data'''
    df = pd.read_csv(csv_path)
    return df

  def _profile_data(self, df:pd.DataFrame):
    '''Check dataset profile, such as missing %, dtype, unique counts, etc'''
    profile = {}
    profile['n_rows'] = len(df)
    profile['n_cols'] = len(df.columns)

    ## Check dtype, missing %,
    col_info = df.describe().to_dict()
    for i in df.columns:
      col_info[i]['missing_pct'] = 1 - col_info[i]['count']/len(df)
      col_info[i]['dtype'] = str(df[i].dtype)
      col_info[i]['unique_count'] = len(df[i].unique())

    profile['col_info'] = col_info

    return profile

  async def _llm_interpretation(self, profile_dict, varb_info_path = False):
    '''
    Sends data summary and variable information if any to LLM
    '''
    if varb_info_path:
      with open(varb_info_path, 'r') as file:
        varb_info = file.read()
    else:
      varb_info = 'None'

    prompt = f"""
    Given the following dataset profile:
    {json.dumps(profile_dict, indent = 2)}
    and the variable information below:
    {varb_info}

    Please :
    1. Identify likely target variable if no target provided in instructions; otherwise, use the provided target
    2. Identify useful predictor features
    3. Identify columns to drop and reasoning
    4. Suggest feature engineering (e.g. log transform, bucketization). \
    Any columns identified as 'drop columns' should not be selected for feature engineering.
    5. Summarize missing variable issues and solutions

    Return JSON structured as:
    {{
      'likely_targets':[],
      'selected_features':[],
      'drop_columns':[{{'Feature','Reason'}}],
      'feature_engineering':[{{'Feature','Method','Reason'}}],
      'missing_value_handling':[{{'Feature','Method','Reason'}}]

    }}

    """

    result = await Runner.run(self.agent, prompt)
    print(result.final_output)
    return result.final_output

  async def generate_transformation_code(self, df_profile, suggestions):
    """
    Based on suggestions, write code to transform data.
    """


    prompt = f"""
    Based on the following:
    Data Profile:\n{json.dumps(df_profile, indent = 2)},
    Data Transformation Suggestions:\n{json.dumps(suggestions, indent = 2)},

    Write python code that:
    1. The dataset is called 'df'. Do not change the name of the dataset. Do not \
    read additional data.
    2. Applies suggested transformations from Suggestions
    3. return the transformed data in a pandas dataframe format as 'df_transformed'

    Only return python code. No explanation.
    """


    result = await Runner.run(self.agent, prompt)
    # print(result.final_output)
    return result.final_output

  def execute_code(self, csv_path: str,  code:str):
    '''Executes generated code and return df_transformed'''
    df = pd.read_csv(csv_path)
    local_varbs = {'df':df.copy(), 'pd':pd, 'np': np}
    code = code.replace("```", "")
    code = code.removeprefix("python")
    exec(code, {}, local_varbs)
    df_transformed = local_varbs.get('df_transformed',None)

    return df_transformed












In [6]:
# ## No target provided
# csv_path = userdata.get('train_data_path')
# varb_info_path=userdata.get('varb_info_path')
# user_instructions = "build a model that predicts excess returns and includes a betting strategy\
#  designed to outperform the S&P 500 while staying within a 120% volatility constraint. We’ll\
#   provide daily data that combines public market information with our proprietary dataset, giving\
#    you the raw material to uncover patterns most miss."
# fea_agent = feature_analysis_Agent(user_instructions = user_instructions)
# result = await fea_agent.run(csv_path = csv_path, varb_info_path=varb_info_path)


In [6]:
## target provided
csv_path = userdata.get('train_data_path')
varb_info_path=userdata.get('varb_info_path')
user_instructions = "build a model that predicts excess returns and includes a betting strategy\
 designed to outperform the S&P 500 while staying within a 120% volatility constraint. We’ll\
  provide daily data that combines public market information with our proprietary dataset, giving\
   you the raw material to uncover patterns most miss."
fea_agent = feature_analysis_Agent(user_instructions = user_instructions, user_defined_target='market_forward_excess_returns')



In [7]:
fea_eng_result = await fea_agent.run(csv_path = csv_path, varb_info_path=varb_info_path)

```json
{
  "likely_targets": [
    "market_forward_excess_returns"
  ],
  "selected_features": [
    "M*", "E*", "I*", "P*", "V*", "S*", "D*"
  ],
  "drop_columns": [
    {
      "Feature": "date_id",
      "Reason": "Identifier column, not useful for prediction"
    },
    {
      "Feature": "forward_returns",
      "Reason": "Not available in the test set"
    },
    {
      "Feature": "risk_free_rate",
      "Reason": "Not available in the test set"
    }
  ],
  "feature_engineering": [
    {
      "Feature": "E*",
      "Method": "Standardization",
      "Reason": "Macro Economic features have different scales"
    },
    {
      "Feature": "I*",
      "Method": "Standardization",
      "Reason": "Interest Rate features have different scales"
    },
    {
      "Feature": "P*",
      "Method": "Log Transform",
      "Reason": "Price/Valuation features may have skewed distributions"
    },
    {
      "Feature": "V*",
      "Method": "Standardization",
      "Reason": "Volatility f

In [8]:
fea_eng_result['Feature_analysis_suggestions']

'```json\n{\n  "likely_targets": [\n    "market_forward_excess_returns"\n  ],\n  "selected_features": [\n    "M*", "E*", "I*", "P*", "V*", "S*", "D*"\n  ],\n  "drop_columns": [\n    {\n      "Feature": "date_id",\n      "Reason": "Identifier column, not useful for prediction"\n    },\n    {\n      "Feature": "forward_returns",\n      "Reason": "Not available in the test set"\n    },\n    {\n      "Feature": "risk_free_rate",\n      "Reason": "Not available in the test set"\n    }\n  ],\n  "feature_engineering": [\n    {\n      "Feature": "E*",\n      "Method": "Standardization",\n      "Reason": "Macro Economic features have different scales"\n    },\n    {\n      "Feature": "I*",\n      "Method": "Standardization",\n      "Reason": "Interest Rate features have different scales"\n    },\n    {\n      "Feature": "P*",\n      "Method": "Log Transform",\n      "Reason": "Price/Valuation features may have skewed distributions"\n    },\n    {\n      "Feature": "V*",\n      "Method": "Standa

In [9]:
code = await fea_agent.generate_transformation_code(
    fea_eng_result['raw_profile'],
    fea_eng_result['Feature_analysis_suggestions']
)

print("Generated code:")
print(code)

Generated code:
```python
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import FunctionTransformer

# Drop unnecessary columns
df = df.drop(columns=['date_id', 'forward_returns', 'risk_free_rate'])

# Handle missing values
for col in df.columns:
    if col.startswith('M'):
        df[col] = df[col].fillna(method='ffill')
    elif col.startswith(('E', 'I', 'P', 'V', 'S')):
        df[col] = df[col].fillna(df[col].mean())

# Standardization
scaler = StandardScaler()
for prefix in ['E', 'I', 'V', 'S']:
    cols = [col for col in df.columns if col.startswith(prefix)]
    df[cols] = scaler.fit_transform(df[cols])

# Log Transform for Price/Valuation features
log_transformer = FunctionTransformer(np.log1p, validate=True)
p_cols = [col for col in df.columns if col.startswith('P')]
df[p_cols] = log_transformer.fit_transform(df[p_cols])

# Return the transformed dataframe
df_transformed = df
```


In [10]:
# Execute to get transformed df
import numpy as np
csv_path = userdata.get('train_data_path')
df_transformed = fea_agent.execute_code(csv_path = csv_path, code = code)

display(df_transformed.head())

<string>:13: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
/usr/local/lib/python3.12/dist-packages/sklearn/preprocessing/_function_transformer.py:387: RuntimeWarning: invalid value encountered in log1p
  return func(X, **(kw_args if kw_args else {}))


,D1,D2,D3,D4,D5,D6,D7,D8,D9,E1,...,V13,V2,V3,V4,V5,V6,V7,V8,V9,market_forward_excess_returns
0,0,0,0,1,1,0,0,0,1,-3.912782e-16,...,1.143207e-17,0.0,1.923667e-16,-3.843260e-16,-5.277907e-17,0.0,0.0,0.0,0.0,-0.003038
1,0,0,0,1,1,0,0,0,1,-3.912782e-16,...,1.143207e-17,0.0,1.923667e-16,-3.843260e-16,-5.277907e-17,0.0,0.0,0.0,0.0,-0.009114
2,0,0,0,1,0,0,0,0,1,-3.912782e-16,...,1.143207e-17,0.0,1.923667e-16,-3.843260e-16,-5.277907e-17,0.0,0.0,0.0,0.0,-0.010243
3,0,0,0,1,0,0,0,0,0,-3.912782e-16,...,1.143207e-17,0.0,1.923667e-16,-3.843260e-16,-5.277907e-17,0.0,0.0,0.0,0.0,0.004046
4,0,0,0,1,0,0,0,0,0,-3.912782e-16,...,1.143207e-17,0.0,1.923667e-16,-3.843260e-16,-5.277907e-17,0.0,0.0,0.0,0.0,-0.012301


In [11]:
# ## Build a coding agent to write code according to feature analysis agent's suggestions
# ## to-do: make it more generic and embedded it into other agents
# class CodingAgent:

#   def __init__(self, model:str = 'gpt-4o'):
#     self.model = model
#     self.name = 'Coding Agent'
#     self.user_instructions = 'You are a coding engineer specialized in data science.\
#     You take suggestions as instructions from the feature analysis agent and write \
#     code to transform the given data. In the end, you return the transformed dataset \
#     in a pandas dataframe format. '
#     self.agent = Agent(
#         name = self.name,
#         model = self.model,
#         instructions = self.user_instructions,
#         model_settings = ModelSettings(temperature = 0)
#         )


#   async def generate_transformation_code(self, df_profile, context):
#     """
#     Take contexts from other agents and generate code to complete their missions.
#     Contexts from other agents are in JSON format
#     """


#     prompt = f"""
#     Based on the following:
#     Data Profile:\n{json.dumps(df_profile, indent = 2)},
#     Agent Contexts:\n{json.dumps(context, indent = 2)},

#     Write python code that:
#     1. The dataset is called 'df'. Do not change the name of the dataset. Do not \
#     read additional data.
#     2. Applies suggested transformations from the Feature Analysis Agent Suggestions
#     3. return the transformed data in a pandas dataframe format as 'df_transformed'

#     Only return python code. No explanation.
#     """


#     result = await Runner.run(self.agent, prompt)
#     # print(result.final_output)
#     return result.final_output

#   def execute_code(self, csv_path: str,  code:str):
#     '''Executes generated code and return df_transformed'''
#     df = pd.read_csv(csv_path)
#     local_varbs = {'df':df.copy(), 'pd':pd, 'np': np}
#     code = code.replace("```", "")
#     code = code.removeprefix("python")
#     exec(code, {}, local_varbs)
#     df_transformed = local_varbs.get('df_transformed',None)

#     return df_transformed


# coding_agent = CodingAgent()


# code = await coding_agent.generate_transformation_code(
#     fea_eng_result['raw_profile'],
#     fea_eng_result['Feature_analysis_suggestions']
# )

# print("Generated code:")
# print(code)

# # Execute to get transformed df
# import numpy as np
# csv_path = '/content/drive/MyDrive/Colab Notebooks/MLAgent/train.csv'

# df_transformed = coding_agent.execute_code(csv_path = csv_path, code = code)

# display(df_transformed.head())

In [21]:
class ModelingAgent:

  def __init__(self, model = 'gpt-4o',user_instructions = '', user_defined_target = ''):
    self.model = model
    self.name = 'Modeling Agent'
    self.user_defined_target = user_defined_target
    self.user_instructions = 'You are a data scientist specialized in machine learning modeling.\
    You take in problem statement, variable descriptions, transformed data based on the suggestions by the feature analysis agent, \
     and target variable(s). Based on these information, you make suggestions \
     on: \
     1. What kind of model we should build (e.g. Classification or regression)\
     2. What machine learning algorithm to use (e.g. linear regression,time series, XGBoost, or deep learning)\
     3. What hyperparameter we should tune\
      '+user_instructions

    if self.user_defined_target:
      self.user_instructions += f'The target variable is {self.user_defined_target}'

    self.agent = Agent(
        name = self.name,
        model = self.model,
        instructions = self.user_instructions,
        model_settings = ModelSettings(temperature = 0)
        )


  async def proposed_model(self, problem_statement:str, varb_info_path:str,fea_eng_suggestions:str, df_transformed:pd.DataFrame, target_variable:str):

    if varb_info_path:
      with open(varb_info_path, 'r') as file:
        original_varb_info = file.read()
    else:
      original_varb_info = 'None'


    schema = {
        'columns':list(df_transformed.columns),
        'problem_statement':problem_statement,
        'original_varb_info':original_varb_info,
        'fea_eng_suggestions':fea_eng_suggestions,
        'target_variable':target_variable
    }

    prompt = f"""
    You are given:
    * Problem Statement: {schema.get('problem_statement')}
    * Original Variable Descriptions: {schema.get('original_varb_info')}
    * Feature Engineering Suggestions: {schema.get('fea_eng_suggestions')}
    * Transformed Data: {schema.get('columns')}
    * Target Variable: {schema.get('target_variable')}

    Tasks:
    1. Determine whetehr problem is regression or classification
    2. Select a machine learning model type
    3. Recommend hyperparameters to tune on
    4. Recommend evaluation protocol (cross validation or train/test split only)
    5. If cross validation, recommend train/validation/test split ratio as 0.7:0.2:0.1. \
    If train/test split only, recommend train/test split ratio as 0.8:0.2.
    6. Suggest metrics to evaluate model performance
    7. Explain reasoning of the above decisions
    8. Return JSON strictly:

    {{
      'target_variable':'{target_variable}',
      'features':{str([x for x in list(df_transformed.columns) if x != target_variable])}.
      'task_type':'regression'|'classification',
      'model_type':'linear_regression'|'time_series'|'xgboost'|'deep_learning'|...,
      'hyperparameters':'hyperparameter':'value',
      'evaluation_protocol':'cross_validation'|'train_test_split_only',
      'train_test_split_ratio':0.7:0.2:0.1 | 0.8:0.2,
      'metrics':['metric1','metric2','metrics3'...],
      'reasoning':'reasoning'


    }}

    """

    result = await Runner.run(self.agent, prompt)
    print(result.final_output)
    return result.final_output

  async def generate_modeling_code(self, modeling_proposal:str,):
    """
    Based on modeling_proposal, write code to train model and evaluate model performance.
    """


    prompt = f"""
    Based on the following:
    Modeling_proposal:\n{json.dumps(modeling_proposal, indent = 2)},

    Write python code that:
    1. The dataset is called 'df_transformed'. Do not change the name of the dataset. Do not \
    read additional data.
    2. Applies modeling proposal from Modeling_proposal
    3. Write code to train model and evaluate model performance. \
        a. If you use 'early_stopping_rounds' as a parameter, pass it to the constructor of model.
        b. Save training history
    4. return model as 'Model',  evaluation result as 'evaluation_result':{{'metric1':float,'metric2':float...}}\
    training history as 'training_history'

    Only return python code. No explanation.
    """


    result = await Runner.run(self.agent, prompt)
    print(result.final_output)
    return result.final_output

  async def execute_code(self, df_transformed: pd.DataFrame,  code:str):
    '''Executes generated code and return model and result'''
    local_varbs = {'df_transformed':df_transformed.copy(), 'pd':pd, 'np': np}
    code = code.replace("```", "")
    code = code.removeprefix("python")
    try:
      exec(code, {}, local_varbs)
    except Exception as e:
      prompt = f'''Receive this error: {e}. Fix the error in the original code.Original code: {code}. Only return python code. No explanation.'''
      result = await Runner.run(self.agent, prompt)
      print(f"{prompt} \n Error: {e} \n Updated code: {result.final_output}")
      code = result.final_output
      code = code.replace("```", "")
      code = code.removeprefix("python")
      exec(code, {}, local_varbs)

    model = local_varbs.get('Model',None)
    evaluation_result = local_varbs.get('evaluation_result',None)
    training_history = local_varbs.get('training_history',None)

    return model, evaluation_result,training_history



In [22]:
modeling_agent=ModelingAgent()


In [23]:
problem_statement = "build a model that predicts excess returns and includes a betting strategy\
 designed to outperform the S&P 500 while staying within a 120% volatility constraint. We’ll\
  provide daily data that combines public market information with our proprietary dataset, giving\
   you the raw material to uncover patterns most miss."

varb_info_path=userdata.get('varb_info_path')


fea_eng_suggestions = fea_eng_result['Feature_analysis_suggestions']
df_transformed = df_transformed.copy()
target_variable = 'market_forward_excess_returns'

In [24]:
model_proposal = await modeling_agent.proposed_model(problem_statement, varb_info_path,fea_eng_suggestions, df_transformed, target_variable)

```json
{
  "target_variable": "market_forward_excess_returns",
  "features": [
    "D1", "D2", "D3", "D4", "D5", "D6", "D7", "D8", "D9", "E1", "E10", "E11", "E12", "E13", "E14", "E15", "E16", "E17", "E18", "E19", "E2", "E20", "E3", "E4", "E5", "E6", "E7", "E8", "E9", "I1", "I2", "I3", "I4", "I5", "I6", "I7", "I8", "I9", "M1", "M10", "M11", "M12", "M13", "M14", "M15", "M16", "M17", "M18", "M2", "M3", "M4", "M5", "M6", "M7", "M8", "M9", "P1", "P10", "P11", "P12", "P13", "P2", "P3", "P4", "P5", "P6", "P7", "P8", "P9", "S1", "S10", "S11", "S12", "S2", "S3", "S4", "S5", "S6", "S7", "S8", "S9", "V1", "V10", "V11", "V12", "V13", "V2", "V3", "V4", "V5", "V6", "V7", "V8", "V9"
  ],
  "task_type": "regression",
  "model_type": "xgboost",
  "hyperparameters": {
    "n_estimators": 100,
    "max_depth": 6,
    "learning_rate": 0.1,
    "subsample": 0.8
  },
  "evaluation_protocol": "cross_validation",
  "train_test_split_ratio": "0.7:0.2:0.1",
  "metrics": ["mean_squared_error", "r2_score"],
  "r

In [32]:
# model_proposal

In [26]:
# df_transformed

In [27]:
modeling_code = await modeling_agent.generate_modeling_code(model_proposal)


```python
import xgboost as xgb
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

# Split the data
train_data, temp_data = train_test_split(df_transformed, test_size=0.3, random_state=42)
val_data, test_data = train_test_split(temp_data, test_size=0.333, random_state=42)

# Separate features and target
X_train = train_data.drop(columns=['market_forward_excess_returns'])
y_train = train_data['market_forward_excess_returns']
X_val = val_data.drop(columns=['market_forward_excess_returns'])
y_val = val_data['market_forward_excess_returns']
X_test = test_data.drop(columns=['market_forward_excess_returns'])
y_test = test_data['market_forward_excess_returns']

# Define the model
model = xgb.XGBRegressor(
    n_estimators=100,
    max_depth=6,
    learning_rate=0.1,
    subsample=0.8,
    early_stopping_rounds=10,
    eval_metric='rmse'
)

# Train the model
eval_set = [(X_train, y_train), (X_val, y

In [28]:
Model, evaluation_result,training_history = await modeling_agent.execute_code(df_transformed,  modeling_code)

In [29]:
evaluation_result

{'mean_squared_error': 0.00012433537352600965,
 'r2_score': -0.007761737315662209}

In [30]:
training_history

{'validation_0': OrderedDict([('rmse',
               [0.01048321417217923,
                0.01042337517059177,
                0.01034180539341818,
                0.01026747199721427,
                0.01018750101130327,
                0.01010324176376356,
                0.01002177966758828,
                0.00997961587603813,
                0.00992030629982711,
                0.00984340918413461,
                0.00975512465657176])]),
 'validation_1': OrderedDict([('rmse',
               [0.01030226987501686,
                0.01030889021918647,
                0.01031911792443535,
                0.01032441459757397,
                0.01033849619172613,
                0.01034570895143787,
                0.01033870291090117,
                0.0103415728491934,
                0.01034034650442424,
                0.01034367542444489,
                0.0103585113603232])])}

In [40]:
## Build an evaluation agent

# to-do


class EvaluationAgent:

  def __init__(self, model = 'gpt-4o',user_instructions = '', user_defined_target = ''):
    self.model = model
    self.name = 'Evaluation Agent'
    self.user_defined_target = user_defined_target
    self.user_instructions = 'You are a data scientist specialized in evaluating machine learning model.\
    You take in modeling proposal, training result, and training history.\
    Based on these information, you make suggestions on how to improve the model\
      '+user_instructions

    if self.user_defined_target:
      self.user_instructions += f'The target variable is {self.user_defined_target}'

    self.agent = Agent(
        name = self.name,
        model = self.model,
        instructions = self.user_instructions,
        model_settings = ModelSettings(temperature = 0)
        )


  async def analyze_model(self, modeling_proposal:str, training_result:str,training_history:str):


    schema = {
        'modeling_proposal':modeling_proposal,
        'training_result':training_result,
        'training_history':training_history,
        'target_variable':self.user_defined_target
    }

    prompt = f"""
    You are given:
    * Modeling Proposal: {schema.get('modeling_proposal')}
    * Training Result: {schema.get('training_result')}
    * Training History: {schema.get('training_history')}
    * Target Variable: {schema.get('target_variable')}

    Tasks:
    1. Read Modeling Proposal
    2. Examinate the training history and training result
    3. Suggest method to improve the model
    4. Explain reasoning of the above decisions
    5. Return JSON strictly:

    {{
      'target_variable':'{self.user_defined_target }',
      'modeling_proposal':{modeling_proposal}.
      'training_result':{training_result},
      'reasoning':'reasoning'


    }}

    """

    result = await Runner.run(self.agent, prompt)
    print(result.final_output)
    return result.final_output

  # async def generate_modeling_code(self, modeling_proposal:str,):
  #   """
  #   Based on modeling_proposal, write code to train model and evaluate model performance.
  #   """


  #   prompt = f"""
  #   Based on the following:
  #   Modeling_proposal:\n{json.dumps(modeling_proposal, indent = 2)},

  #   Write python code that:
  #   1. The dataset is called 'df_transformed'. Do not change the name of the dataset. Do not \
  #   read additional data.
  #   2. Applies modeling proposal from Modeling_proposal
  #   3. Write code to train model and evaluate model performance. \
  #   If you use 'early_stopping_rounds' as a parameter, pass it to the constructor of model.
  #   4. return model as 'Model' and evaluation result as 'evaluation_result':{{'metric1':float,'metric2':float...}}

  #   Only return python code. No explanation.
  #   """


  #   result = await Runner.run(self.agent, prompt)
  #   print(result.final_output)
  #   return result.final_output

  # def execute_code(self, df_transformed: pd.DataFrame,  code:str):
  #   '''Executes generated code and return model and result'''
  #   local_varbs = {'df_transformed':df_transformed.copy(), 'pd':pd, 'np': np}
  #   code = code.replace("```", "")
  #   code = code.removeprefix("python")
  #   exec(code, {}, local_varbs)
  #   model = local_varbs.get('Model',None)
  #   evaluation_result = local_varbs.get('evaluation_result',None)

  #   return model, evaluation_result





In [49]:
evaluation_agent = EvaluationAgent(user_instructions = problem_statement,user_defined_target =target_variable)

In [52]:
evaluator_suggestion = await evaluation_agent.analyze_model(model_proposal, evaluation_result,training_history)

```json
{
  "target_variable": "market_forward_excess_returns",
  "modeling_proposal": {
    "target_variable": "market_forward_excess_returns",
    "features": [
      "D1", "D2", "D3", "D4", "D5", "D6", "D7", "D8", "D9", "E1", "E10", "E11", "E12", "E13", "E14", "E15", "E16", "E17", "E18", "E19", "E2", "E20", "E3", "E4", "E5", "E6", "E7", "E8", "E9", "I1", "I2", "I3", "I4", "I5", "I6", "I7", "I8", "I9", "M1", "M10", "M11", "M12", "M13", "M14", "M15", "M16", "M17", "M18", "M2", "M3", "M4", "M5", "M6", "M7", "M8", "M9", "P1", "P10", "P11", "P12", "P13", "P2", "P3", "P4", "P5", "P6", "P7", "P8", "P9", "S1", "S10", "S11", "S12", "S2", "S3", "S4", "S5", "S6", "S7", "S8", "S9", "V1", "V10", "V11", "V12", "V13", "V2", "V3", "V4", "V5", "V6", "V7", "V8", "V9"
    ],
    "task_type": "regression",
    "model_type": "xgboost",
    "hyperparameters": {
      "n_estimators": 100,
      "max_depth": 6,
      "learning_rate": 0.1,
      "subsample": 0.8
    },
    "evaluation_protocol": "cross_vali